In [0]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [0]:
# ! cp data/Train_target.csv ./raw_data/Train_target.csv
# ! cp data/Test_target.csv ./raw_data/Test_target.csv

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train=pd.read_csv("/content/drive/My Drive/Data_Science/Zindi_Competitions/Zindi_Airquo/Airquo_resources/Train.csv")
test=pd.read_csv("/content/drive/My Drive/Data_Science/Zindi_Competitions/Zindi_Airquo/Airquo_resources/Test.csv")

In [0]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [0]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"23.96666667,22.88333333,22.28333333,21.4833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.6898333329999999,0.722333333,0.7755833329999...","290.1352379,328.47011289999995,323.5730796,282...","0.6333333329999999,0.725,0.955,0.8725,0.6925,0...","87.64333333,87.67,87.70583333,87.75083333,87.7..."
1,ID_test_1,C,"18.99166667,19.70833333,20.95833333,22.7833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.99975,0.99725,0.956333333,0.851583333,0.7775...","295.2350543,307.6569135,313.7360236,311.518385...","0.640833333,1.244166667,1.284166667,1.2875,1.3...","87.8925,87.95666667,87.9925,87.9925,87.9808333..."
2,ID_test_10,D,"25.11666667,25.08333333,24.40833333,23.575,22....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.760166667,0.733583333,0.78475,0.825833333,0....","77.06161145,95.48124997,61.44498258,89.4688360...","2.0133333330000003,1.3025,0.783333333,0.530833...","90.34,90.2975,90.305,90.35583333,90.4425,90.49..."
3,ID_test_100,C,"18.71666667,20.60833333,22.28333333,23.9833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.604,0.0,0.0,...","0.951416667,0.92575,0.8603333329999999,0.77883...","310.7509058,316.186793,324.8167326,327.2174992...","0.745,1.169166667,1.316666667,1.385,1.40083333...","87.98833333,88.06333333,88.07833333,88.0366666..."
4,ID_test_1000,D,"17.63333333,18.05833333,19.625,21.28333333,23....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.9,0.898333333,0.8725,0.831666667,0.820833332...","334.6054069,329.6848561000001,322.7668036,312....","0.620833333,0.5,0.456666667,0.558333333,1.0158...","90.515,90.5575,90.62333333,90.6825,90.67916667..."


In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

### Features engineering part

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [0]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

# Airqo Default Feature Engr

In [0]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.43it/s]


In [0]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

  0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
data_p = data.drop(features,1)

In [0]:
data_p.describe()

,target,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
count,15539.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.0,20574.000000,20574.000000,20574.000000,20574.0,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000,20574.000000
mean,58.242429,29.392256,17.505479,22.657246,3.210024,10.790403,21.954074,11.886777,8.182841,0.0,0.170715,0.922876,1.893995,0.0,8.182841,0.976105,0.514398,0.799901,0.126985,0.016920,0.828865,0.461707,354.304447,4.446200,200.419005,106.915547,11884.642309,210.912507,349.858247,3.012514,0.331061,1.056581,0.558650,0.355425,0.911180,2.681454,89.284593,88.644200,88.980748,0.146042,0.021740,88.989008,0.640394
std,42.373700,2.120139,1.079206,1.283776,0.697262,4.653355,1.397183,2.390384,9.165404,0.0,0.213622,1.020953,3.584665,0.0,9.165404,0.031340,0.096861,0.071501,0.028199,0.007219,0.080634,0.086626,20.907216,8.419156,31.565978,21.300942,4635.460449,63.606790,23.582052,1.904345,0.441071,0.568914,0.208177,0.394869,0.562593,1.840857,1.088453,1.039634,1.068988,0.020285,0.006175,1.071085,0.106741
min,1.452619,19.733333,13.783333,18.592222,0.554706,0.307698,18.541667,2.133333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.794167,0.172417,0.521925,0.022262,0.000496,0.503333,0.076348,161.668792,0.012510,35.228166,25.696241,660.296815,25.032218,141.753011,0.823333,0.123333,0.438386,0.130991,0.017159,0.393333,0.509167,87.871667,87.250000,87.605234,0.069121,0.004778,87.597500,0.239167
25%,33.482625,27.781818,16.783333,21.570231,2.717366,7.384079,20.791667,10.150000,0.833000,0.0,0.013769,0.094493,0.008929,0.0,0.833000,0.961083,0.442500,0.757275,0.106614,0.011366,0.780288,0.401000,355.456072,0.685438,181.006943,94.228396,8878.990551,170.586440,349.904613,2.099167,0.222500,0.810020,0.431069,0.185821,0.672917,1.845833,88.606667,87.989091,88.325114,0.131928,0.017405,88.338333,0.570000
50%,46.504048,29.383333,17.483333,22.575237,3.168595,10.039994,21.866667,11.608333,5.472000,0.0,0.102752,0.623129,0.388290,0.0,5.472000,0.990000,0.520000,0.805881,0.127197,0.016179,0.839167,0.459000,358.196966,1.911985,199.793825,105.139027,11054.214981,198.002499,354.876629,2.626667,0.259167,0.921759,0.523383,0.273930,0.762500,2.364167,88.740000,88.168333,88.458161,0.144835,0.020977,88.465833,0.637500
75%,68.569062,30.741667,18.241667,23.579610,3.681772,13.555443,23.041667,13.575000,11.640000,0.0,0.249897,1.348988,1.819767,0.0,11.640000,0.999583,0.586500,0.855071,0.146609,0.021494,0.887500,0.525083,359.398357,4.820115,218.831171,122.318287,14961.763364,260.701162,357.726376,3.366667,0.297500,1.095665,0.641866,0.411992,0.940000,3.062500,90.798333,90.057500,90.460904,0.158317,0.025064,90.469167,0.701667
max,475.820000,36.533333,20.600000,27.549449,5.704088,32.536622,26.625000,18.933333,45.833000,0.0,2.310450,5.636605,31.771315,0.0,45.833000,1.000000,0.897818,0.944675,0.214803,0.046141,0.997167,0.755583,359.997383,150.228959,315.085561,165.172923,27282.094417,343.774871,359.868346,24.866667,4.543333,5.733605,2.590533,6.710864,5.613333,24.659167,91.130833,90.503333,90.807583,0.269699,0.072738,90.817500,1.033333


In [0]:
data_p[data_p.columns[0:]].corr()['target'][:]

target                 1.000000
max_temp              -0.133647
min_temp              -0.156444
mean_temp             -0.181266
std_temp              -0.052610
var_temp              -0.046772
median_temp           -0.160509
ptp_temp              -0.050680
max_precip             0.044026
min_precip                  NaN
mean_precip            0.012927
std_precip             0.034709
var_precip             0.039156
median_precip               NaN
ptp_precip             0.044026
max_rel_humidity       0.109942
min_rel_humidity       0.063361
mean_rel_humidity      0.112319
std_rel_humidity      -0.003227
var_rel_humidity      -0.014480
median_rel_humidity    0.118092
ptp_rel_humidity      -0.031066
max_wind_dir           0.035353
min_wind_dir           0.015409
mean_wind_dir          0.171200
std_wind_dir           0.002084
var_wind_dir          -0.002710
median_wind_dir        0.112039
ptp_wind_dir           0.028282
max_wind_spd          -0.054588
min_wind_spd          -0.069410
mean_win

In [0]:
data.shape

(20574, 9)

In [0]:
data.describe()

,target
count,15539.000000
mean,58.242429
std,42.373700
min,1.452619
25%,33.482625
50%,46.504048
75%,68.569062
max,475.820000


In [0]:
data['atmos_press'].head()

0    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
1    [90.32, 90.3775, 90.44083333, 90.4725, 90.4541...
2    [88.55166667, 88.46416667, 88.31916667, 88.24,...
3    [88.615, 88.53083333, 88.4, 88.27166667, 88.20...
4    [88.55666667, 88.64083333, 88.65833333, 88.647...
Name: atmos_press, dtype: object

In [0]:
for x in range(100):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [0]:
data.shape

(20574, 609)

In [0]:
data['newrel_humidity120'].head()

0    0.874583
1    0.797333
2    0.437917
3    0.719667
4         NaN
Name: newrel_humidity120, dtype: float64

In [0]:
data.shape

(20574, 735)

In [0]:
len(data.temp[20])

121

In [0]:
len(data.precip[10])

121

In [0]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.73it/s]


In [0]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:20<00:00,  3.37s/it]


In [0]:
data.drop(features,1,inplace=True)

In [0]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [0]:
train.describe()

,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,atmospherepressure7,newtemp8,newprecip8,newrel_humidity8,newwind_dir8,windspeed8,atmospherepressure8,newtemp9,newprecip9,newrel_humidity9,newwind_dir9,windspeed9,atmospherepressure9,newtemp10,newprecip10,newrel_humidity10,newwind_dir10,windspeed10,atmospherepressure10,newtemp11,newprecip11,newrel_humidity11,newwind_dir11,windspeed11,atmospherepressure11,newtemp12,newprecip12,newrel_humidity12,newwind_dir12,windspeed12,atmospherepressure12,newtemp13,newprecip13,newrel_humidity13,newwind_dir13,windspeed13,atmospherepressure13,newtemp14,newprecip14,newrel_humidity14,newwind_dir14,windspeed14,atmospherepressure14,newtemp15,newprecip15,newrel_humidity15,newwind_dir15,windspeed15,atmospherepressure15,newtemp16,newprecip16,newrel_humidity16,...,newrel_humidity111,newwind_dir111,windspeed111,atmospherepressure111,newtemp112,newprecip112,newrel_humidity112,newwind_dir112,windspeed112,atmospherepressure112,newtemp113,newprecip113,newrel_humidity113,newwind_dir113,windspeed113,atmospherepressure113,newtemp114,newprecip114,newrel_humidity114,newwind_dir114,windspeed114,atmospherepressure114,newtemp115,newprecip115,newrel_humidity115,newwind_dir115,windspeed115,atmospherepressure115,newtemp116,newprecip116,newrel_humidity116,newwind_dir116,windspeed116,atmospherepressure116,newtemp117,newprecip117,newrel_humidity117,newwind_dir117,windspeed117,atmospherepressure117,newtemp118,newprecip118,newrel_humidity118,newwind_dir118,windspeed118,atmospherepressure118,newtemp119,newprecip119,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
count,15539.000000,14486.000000,14453.000000,14491.000000,13795.000000,13795.000000,14528.000000,14494.000000,14463.000000,14499.000000,13809.000000,13809.000000,14537.000000,14499.000000,14470.000000,14506.000000,13818.000000,13818.000000,14543.000000,14508.000000,14481.000000,14516.000000,13831.000000,13831.000000,14553.000000,14512.000000,14486.000000,14520.000000,13839.000000,13839.000000,14557.000000,14520.000000,14494.000000,14528.000000,13849.000000,13849.000000,14564.000000,14524.000000,14499.000000,14532.000000,13856.000000,13856.000000,14568.000000,14528.000000,14504.000000,14537.000000,13863.000000,13863.000000,14572.000000,14530.000000,14507.000000,14538.000000,13869.000000,13869.000000,14574.000000,14534.000000,14512.000000,14543.000000,13878.000000,13878.000000,14578.000000,14536.000000,14515.000000,14546.000000,13886.000000,13886.000000,14580.000000,14538.000000,14518.000000,14547.000000,13889.000000,13889.000000,14582.000000,14538.000000,14518.000000,14546.000000,13888.000000,13888.000000,14581.000000,14540.000000,14520.000000,14546.000000,13885.000000,13885.000000,14582.000000,14539.000000,14519.000

In [0]:
test.describe()

,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,atmospherepressure7,newtemp8,newprecip8,newrel_humidity8,newwind_dir8,windspeed8,atmospherepressure8,newtemp9,newprecip9,newrel_humidity9,newwind_dir9,windspeed9,atmospherepressure9,newtemp10,newprecip10,newrel_humidity10,newwind_dir10,windspeed10,atmospherepressure10,newtemp11,newprecip11,newrel_humidity11,newwind_dir11,windspeed11,atmospherepressure11,newtemp12,newprecip12,newrel_humidity12,newwind_dir12,windspeed12,atmospherepressure12,newtemp13,newprecip13,newrel_humidity13,newwind_dir13,windspeed13,atmospherepressure13,newtemp14,newprecip14,newrel_humidity14,newwind_dir14,windspeed14,atmospherepressure14,newtemp15,newprecip15,newrel_humidity15,newwind_dir15,windspeed15,atmospherepressure15,newtemp16,newprecip16,newrel_humidity16,...,newrel_humidity111,newwind_dir111,windspeed111,atmospherepressure111,newtemp112,newprecip112,newrel_humidity112,newwind_dir112,windspeed112,atmospherepressure112,newtemp113,newprecip113,newrel_humidity113,newwind_dir113,windspeed113,atmospherepressure113,newtemp114,newprecip114,newrel_humidity114,newwind_dir114,windspeed114,atmospherepressure114,newtemp115,newprecip115,newrel_humidity115,newwind_dir115,windspeed115,atmospherepressure115,newtemp116,newprecip116,newrel_humidity116,newwind_dir116,windspeed116,atmospherepressure116,newtemp117,newprecip117,newrel_humidity117,newwind_dir117,windspeed117,atmospherepressure117,newtemp118,newprecip118,newrel_humidity118,newwind_dir118,windspeed118,atmospherepressure118,newtemp119,newprecip119,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
count,0.0,4832.000000,4825.000000,4853.000000,4669.000000,4669.000000,4858.000000,4832.000000,4824.000000,4852.000000,4668.000000,4668.000000,4857.000000,4832.000000,4823.000000,4853.000000,4667.000000,4667.000000,4856.000000,4832.000000,4822.000000,4852.000000,4666.000000,4666.000000,4855.000000,4833.000000,4823.000000,4853.000000,4665.000000,4665.000000,4856.000000,4835.000000,4825.000000,4854.000000,4663.000000,4663.000000,4858.000000,4834.000000,4824.000000,4853.000000,4658.000000,4658.000000,4857.000000,4832.000000,4822.000000,4851.000000,4655.000000,4655.000000,4855.000000,4832.000000,4822.000000,4851.000000,4649.000000,4649.000000,4855.000000,4834.000000,4824.000000,4853.000000,4646.000000,4646.000000,4857.000000,4834.000000,4824.000000,4853.000000,4637.000000,4637.000000,4857.000000,4832.000000,4823.000000,4852.000000,4626.000000,4626.000000,4856.000000,4830.000000,4823.000000,4852.000000,4618.000000,4618.000000,4856.000000,4826.000000,4821.000000,4850.000000,4613.000000,4613.000000,4854.000000,4825.000000,4822.000000,4851.000000,4612.000000,4612.000000,4855.000000,4826.000000,4824.000000,4853.000000,4613.00

In [0]:
del data  
gc.collect()

0

In [0]:
train.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,...,windspeed98,atmospherepressure98,newtemp99,newprecip99,newrel_humidity99,newwind_dir99,windspeed99,atmospherepressure99,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.891500,0.510818,0.758530,0.120234,0.014456,0.806292,0.380682,345.726333,6.535299,209.574655,120.231802,14455.686315,220.039988,339.191034,1.517500,0.162500,0.677348,0.394041,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500
1,ID_train_1,D,79.131702,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,0.808083,89.156293,1.126667,90.377500,20.833333,0.0,0.911167,81.968539,0.700833,90.440833,20.983333,0.000,0.916333,291.018632,0.341667,90.472500,20.875000,0.000,0.929750,279.391524,0.440833,90.454167,20.141667,0.0,0.954000,158.026892,0.380000,90.394167,19.375000,0.0,0.964083,248.316596,0.467500,90.333333,18.791667,0.034,0.970750,211.143363,0.4625,...,0.384167,90.563333,22.600,0.000,0.822917,149.349174,0.566667,90.646667,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,0.007025,0.051706,0.002673,0.0,0.561,0.979250,0.403083,0.758058,0.160952,0.025906,0.791000,0.576167,357.730286,3.543198,177.109855,96.802929,9370.807133,153.164843,354.187088,3.009167,0.248333,0.899208,0.539199,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333
2,ID_train_10,A,32.661304,28.975000,0.000,0.573333,NaN,NaN,88.551667,27.950000,0.0,0.597167,NaN,NaN,88.464167,29.600000,0.0,0.566833,NaN,NaN,88.319167,26.425000,0.102,0.627667,NaN,NaN,88.240000,22.091667,0.136,0.755417,NaN,NaN,88.230000,21.775000,0.0,0.777417,NaN,NaN,88.268333,22.333333,0.0,0.787000,NaN,NaN,88.370000,21.950000,2.160,0.831917,NaN,NaN,...,1.738333,88.161667,31.675,0.000,0.376750,242.622640,1.773333,88.059167,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,0.151767,0.798401,0.637445,0.0,7.804,0.930333,0.376750,0.712944,0.168129,0.028267,0.766458,0.553583,357.265773,0.581510,201.373211,92.105195,8483.367004,208.901301,356.684263,2.751667,0.331667,1.038656,0.585221,0.342484,0.942500,2.420000,88.813333,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833
3,ID_train_100,A,53.850238,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,0.790250,293.676960,1.192500,88.530833,25.275000,0.0,0.737500,294.517465,1.324167,88.400000,25.625000,0.000,0.728000,301.921417,1.544167,88.271667,25.866667,0.000,0.704917,334.568073,1.915833,88.207500,25.0916

In [0]:
train.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,...,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.891500,0.510818,0.758530,0.120234,0.014456,0.806292,0.380682,345.726333,6.535299,209.574655,120.231802,14455.686315,220.039988,339.191034,1.517500,0.162500,0.677348,0.394041,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500
1,ID_train_1,D,79.131702,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,0.808083,89.156293,1.126667,90.377500,20.833333,0.0,0.911167,81.968539,0.700833,90.440833,20.983333,0.000,0.916333,291.018632,0.341667,90.472500,20.875000,0.000,0.929750,279.391524,0.440833,90.454167,20.141667,0.0,0.954000,158.026892,0.380000,90.394167,19.375000,...,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,0.007025,0.051706,0.002673,0.0,0.561,0.979250,0.403083,0.758058,0.160952,0.025906,0.791000,0.576167,357.730286,3.543198,177.109855,96.802929,9370.807133,153.164843,354.187088,3.009167,0.248333,0.899208,0.539199,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333
2,ID_train_10,A,32.661304,28.975000,0.000,0.573333,NaN,NaN,88.551667,27.950000,0.0,0.597167,NaN,NaN,88.464167,29.600000,0.0,0.566833,NaN,NaN,88.319167,26.425000,0.102,0.627667,NaN,NaN,88.240000,22.091667,0.136,0.755417,NaN,NaN,88.230000,21.775000,0.0,0.777417,NaN,NaN,88.268333,22.333333,...,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,0.151767,0.798401,0.637445,0.0,7.804,0.930333,0.376750,0.712944,0.168129,0.028267,0.766458,0.553583,357.265773,0.581510,201.373211,92.105195,8483.367004,208.901301,356.684263,2.751667,0.331667,1.038656,0.585221,0.342484,0.942500,2.420000,88.813333,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833
3,ID_train_100,A,53.850238,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,0.790250,293.676960,1.192500,88.530833,25.275000,0.0,0.737500,294.517465,1.324167,88.400000,25.625000,0.000,0.728000,301.921417,1.544167,88.271667,25.866667,0.000,0.704917,334.568073,1.915833,88.207500,25.091667,0.0,0.741167,319.576411,1.840000,88.178333,24.025000,...,20.845273,3.080140,9.487259,19.541667,10.550000,25.787,0.0,1.127273,3.739886,13.986750,0.0,25.787,0.999917,0.618750,0.898326,0.114207,0.013043,0.970167,0.381167,357.246718,3.092044,259.973977,114.240190,13050.821125,318.539747,354.154674,2.744167,0.275000,1.365202,0.580259,0.336701,1.388333,2.469167,88.685000,87.965000,88.376915,0.161758,0.026166,88.400000,0.720000
4,ID_train_1000,A,177.418750,21.875000,0.000,0.856417,21.839974,0.197500,88.556667,21.575000,0.0,0.874917,17.054053,0.244167,88.640833,21.525000,0.0,0.879833,89.264060,0.411667,88.658333,21.433333,0.000,0.860167,123.585424,0.560000,88.647500,20.508333,0.000,0.898000,

In [0]:
test.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,...,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_test_0,C,NaN,23.966667,0.0,0.689833,290.135238,0.633333,87.643333,22.883333,0.0,0.722333,328.470113,0.725000,87.670000,22.283333,0.0,0.775583,323.573080,0.955000,87.705833,21.483333,0.0,0.838417,282.408045,0.872500,87.750833,20.908333,0.0,0.847500,278.460873,0.692500,87.791667,19.908333,0.0,0.859500,243.556262,0.347500,87.793333,19.141667,...,21.389532,2.731833,7.462909,20.908333,9.808333,5.728,0.0,0.204165,0.880254,0.774846,0.0,5.728,0.999833,0.586083,0.842947,0.124179,0.015420,0.859250,0.413750,357.223551,7.092226,231.104760,83.856214,7031.864555,217.738239,350.131326,1.915833,0.171667,0.812555,0.392783,0.154279,0.767500,1.744167,88.020833,87.474167,87.748506,0.132764,0.017626,87.752500,0.546667
1,ID_test_1,C,NaN,18.991667,0.0,0.999750,295.235054,0.640833,87.892500,19.708333,0.0,0.997250,307.656913,1.244167,87.956667,20.958333,0.0,0.956333,313.736024,1.284167,87.992500,22.783333,0.0,0.851583,311.518385,1.287500,87.992500,24.325000,0.0,0.777583,319.699892,1.310833,87.980833,25.016667,0.0,0.735833,324.856397,1.382500,87.928333,25.541667,...,21.302824,2.289268,5.240748,20.716667,9.716667,15.831,0.0,0.208719,1.517969,2.304229,0.0,15.831,0.999750,0.583167,0.830610,0.106384,0.011318,0.852000,0.416583,351.333059,6.290985,212.924101,91.254295,8327.346424,181.691165,345.042074,2.099167,0.210000,0.852562,0.456733,0.208605,0.771667,1.889167,88.181667,87.572500,87.936963,0.113756,0.012940,87.928333,0.609167
2,ID_test_10,D,NaN,25.116667,0.0,0.760167,77.061611,2.013333,90.340000,25.083333,0.0,0.733583,95.481250,1.302500,90.297500,24.408333,0.0,0.784750,61.444983,0.783333,90.305000,23.575000,0.0,0.825833,89.468836,0.530833,90.355833,22.350000,0.0,0.869750,298.164435,0.467500,90.442500,21.525000,0.0,0.906500,11.946559,0.409167,90.493333,21.266667,...,22.144421,2.943047,8.661528,21.458333,12.741667,1.327,0.0,0.040190,0.164596,0.027092,0.0,1.327,0.986750,0.515167,0.868258,0.096427,0.009298,0.903750,0.471583,356.508481,2.279896,167.731590,98.024235,9608.750685,144.225803,354.228585,3.127500,0.237500,0.803140,0.569737,0.324600,0.567500,2.890000,90.785000,90.115000,90.521357,0.136316,0.018582,90.536667,0.670000
3,ID_test_100,C,NaN,18.716667,0.0,0.951417,310.750906,0.745000,87.988333,20.608333,0.0,0.925750,316.186793,1.169167,88.063333,22.283333,0.0,0.860333,324.816733,1.316667,88.078333,23.983333,0.0,0.778833,327.217499,1.385000,88.036667,24.958333,0.0,0.721667,335.828966,1.400833,87.976667,26.050000,0.0,0.674833,333.178301,1.285833,87.873333,26.633333,...,22.361639,2.480003,6.150415,21.616667,8.691667,3.604,0.0,0.035686,0.328542,0.107940,0.0,3.604,0.983250,0.582583,0.792134,0.100795,0.010160,0.812000,0.400667,353.300228,0.125888,199.467348,106.904504,11428.572983,172.913664,353.174340,1.790000,0.215000,0.797528,0.430546,0.185370,0.685000,1.575000,88.078333,87.560000,87.754848,0.125583,0.015771,87.726667,0.518333
4,ID_test_1000,D,NaN,17.633333,0.0,0.900000,334.605407,0.620833,90.5150

In [0]:
train.to_csv("/content/drive/My Drive/Data_Science/Zindi_Airquo/Airquo_resources/trainaway.csv",index  = False)
test.to_csv("/content/drive/My Drive/Data_Science/Zindi_Airquo/Airquo_resources/testaway.csv",index  = False)

In [0]:
train.shape

(15539, 645)

In [0]:
train['ID'].nunique()

15539

In [0]:
test.shape

(5035, 645)

In [0]:
test['ID'].nunique()

5035

# Training catboost

In [0]:
train = train.fillna(train.median())

In [0]:
test = test.fillna(test.median())

In [0]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [0]:
lab.fit(train["location"])
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [0]:
!pip install catboost

     |████████████████████████████████| 64.8MB 65kB/s 


In [0]:
X=train.drop(["ID",'target',],axis =1)
y = train.target

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor( n_estimators=1000,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True,)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.2597779	test: 40.2597779	test1: 41.0396951	best: 41.0396951 (0)	total: 205ms	remaining: 3m 24s
100:	learn: 23.2629952	test: 23.2629952	test1: 27.2339077	best: 27.2256580 (99)	total: 20.7s	remaining: 3m 4s
200:	learn: 19.1985815	test: 19.1985815	test1: 26.3505988	best: 26.3344517 (187)	total: 41s	remaining: 2m 43s
300:	learn: 16.2535090	test: 16.2535090	test1: 25.5967814	best: 25.5967814 (300)	total: 1m 1s	remaining: 2m 22s
400:	learn: 14.0139428	test: 14.0139428	test1: 25.2911164	best: 25.2911164 (400)	total: 1m 21s	remaining: 2m 2s
500:	learn: 12.1733845	test: 12.1733845	test1: 25.0843021	best: 25.0770361 (490)	total: 1m 42s	remaining: 1m 41s
600:	learn: 10.6726333	test: 10.6726333	test1: 24.9667496	best: 24.9627355 (599)	total: 2m 2s	remaining: 1m 21s
700:	learn: 9.4581879	test: 9.4581879	test1: 24.8836229	best: 24.8836229 (700)	total: 2m 22s	remaining: 1m
800:	learn: 8.3972659	test: 8.3972659	test1: 24.6601868	best: 24.6601868 (800)	total: 2m 43s	remaining: 40.6s
900:	l

25.126769837953184

In [0]:
np.mean(errcb2)

25.126769837953184

In [0]:
np.mean(errcb2)

25.00495052195609

In [0]:
np.mean(errcb2)

24.696071129797243

In [0]:
np.mean(errcb2)

24.74667939530182

In [0]:
np.mean(errcb2)

24.73933658152334

In [0]:
np.mean(errcb2)

25.300875517347386

In [0]:
d = {'ID': test.ID, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.target.min(),sub.target.max()

(13.074159805669032, 246.3581059276798)

In [0]:
sub.target.min(),sub.target.max()

(12.066521540145228, 236.28511814973393)

In [0]:
sub.target.min(),sub.target.max()

(13.679604901551434, 247.55836790912963)

In [0]:
sub.target.min(),sub.target.max()

(13.403835323212785, 232.26227882202744)

In [0]:
sub.target.min(),sub.target.max()

(14.077534529184444, 256.4151598809758)

In [0]:
sub.target.min(),sub.target.max()

(15.027919285421117, 256.07518806091645)

In [0]:
sub.to_csv("med_range2.csv",index  = False)